# Modelado Analítico: Star Schema

Este notebook demuestra cómo crear y trabajar con un modelo analítico tipo Star Schema.

**Referencia:** [Modelado Analítico](../modelado/modelado-analitico.md)

**Objetivos:**
- Crear tablas de hechos y dimensiones
- Visualizar estructura del modelo
- Consultar datos usando el modelo

## 1. Importar librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print("✅ Librerías importadas")

## 2. Crear tablas de dimensiones

In [ ]:
# Dimension: Tiempo
fechas = pd.date_range('2024-01-01', '2024-12-31', freq='D')
dim_tiempo = pd.DataFrame({
    'fecha_id': range(1, len(fechas) + 1),
    'fecha': fechas,
    'año': fechas.year,
    'mes': fechas.month,
    'dia_semana': fechas.dayofweek,
    'trimestre': fechas.quarter,
    'es_fin_semana': fechas.dayofweek.isin([5, 6])
})

print(f"✅ Dimension Tiempo creada: {len(dim_tiempo)} registros")
dim_tiempo.head()

In [ ]:
# Dimension: Producto
productos = ['Producto A', 'Producto B', 'Producto C', 'Producto D', 'Producto E']
categorias = ['Electrónica', 'Ropa', 'Hogar', 'Deportes', 'Libros']

dim_producto = pd.DataFrame({
    'producto_id': range(1, len(productos) + 1),
    'nombre': productos,
    'categoria': categorias,
    'precio_base': [100, 50, 75, 120, 25]
})

print(f"✅ Dimension Producto creada: {len(dim_producto)} registros")
dim_producto

In [ ]:
# Dimension: Cliente
clientes = [f'Cliente {i}' for i in range(1, 11)]
regiones = np.random.choice(['Norte', 'Sur', 'Este', 'Oeste'], 10)

dim_cliente = pd.DataFrame({
    'cliente_id': range(1, 11),
    'nombre': clientes,
    'region': regiones,
    'tipo': np.random.choice(['Premium', 'Regular'], 10)
})

print(f"✅ Dimension Cliente creada: {len(dim_cliente)} registros")
dim_cliente.head()

## 3. Crear tabla de hechos (Fact Table)

In [ ]:
# Tabla de hechos: Ventas
np.random.seed(42)
n_ventas = 1000

fact_ventas = pd.DataFrame({
    'venta_id': range(1, n_ventas + 1),
    'fecha_id': np.random.choice(dim_tiempo['fecha_id'], n_ventas),
    'producto_id': np.random.choice(dim_producto['producto_id'], n_ventas),
    'cliente_id': np.random.choice(dim_cliente['cliente_id'], n_ventas),
    'cantidad': np.random.randint(1, 10, n_ventas),
    'precio_unitario': np.random.uniform(50, 150, n_ventas),
    'descuento': np.random.uniform(0, 0.2, n_ventas)
})

# Calcular total
fact_ventas['total'] = fact_ventas['cantidad'] * fact_ventas['precio_unitario'] * (1 - fact_ventas['descuento'])

print(f"✅ Fact Table Ventas creada: {len(fact_ventas)} registros")
fact_ventas.head()

## 4. Visualizar estructura del modelo

In [ ]:
print("=" * 60)
print("ESTRUCTURA STAR SCHEMA")
print("=" * 60)
print("\n        Dimension: Tiempo")
print("              |")
print("              |")
print("    Dimension: Producto --- Fact: Ventas --- Dimension: Cliente")
print("              |")
print("              |")
print("        Dimension: Cliente")
print("\n" + "=" * 60)
print(f"\nFact Table: {len(fact_ventas)} registros")
print(f"Dimension Tiempo: {len(dim_tiempo)} registros")
print(f"Dimension Producto: {len(dim_producto)} registros")
print(f"Dimension Cliente: {len(dim_cliente)} registros")

## 5. Consultar datos usando el modelo

In [ ]:
# JOIN para análisis completo
ventas_completo = fact_ventas.merge(
    dim_tiempo[['fecha_id', 'fecha', 'mes', 'trimestre']], 
    on='fecha_id'
).merge(
    dim_producto[['producto_id', 'nombre', 'categoria']], 
    on='producto_id'
).merge(
    dim_cliente[['cliente_id', 'nombre', 'region', 'tipo']], 
    on='cliente_id'
)

print("✅ Datos combinados (simulando JOIN SQL)")
ventas_completo.head()

## 6. Análisis usando el modelo

In [ ]:
# Ventas por categoría
ventas_categoria = ventas_completo.groupby('categoria')['total'].agg(['sum', 'mean', 'count']).round(2)
ventas_categoria.columns = ['Total', 'Promedio', 'Cantidad']
print("=== VENTAS POR CATEGORÍA ===")
print(ventas_categoria)

In [ ]:
# Ventas por trimestre
ventas_trimestre = ventas_completo.groupby('trimestre')['total'].sum()
print("\n=== VENTAS POR TRIMESTRE ===")
print(ventas_trimestre)

# Visualizar
plt.figure(figsize=(10, 6))
ventas_trimestre.plot(kind='bar', color='steelblue')
plt.title('Ventas por Trimestre')
plt.xlabel('Trimestre')
plt.ylabel('Total Ventas (€)')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# Ventas por región y tipo de cliente
ventas_region_tipo = ventas_completo.groupby(['region', 'tipo'])['total'].sum().unstack(fill_value=0)
print("\n=== VENTAS POR REGIÓN Y TIPO ===")
print(ventas_region_tipo)

# Visualizar
plt.figure(figsize=(10, 6))
ventas_region_tipo.plot(kind='bar', stacked=False)
plt.title('Ventas por Región y Tipo de Cliente')
plt.xlabel('Región')
plt.ylabel('Total Ventas (€)')
plt.legend(title='Tipo Cliente')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

## 7. Ventajas del Star Schema

El Star Schema tiene varias ventajas que lo hacen ideal para analytics:

In [ ]:
print("=" * 60)
print("VENTAJAS DEL STAR SCHEMA")
print("=" * 60)
print("\n✅ Fácil de entender para usuarios de negocio")
print("✅ Consultas más rápidas (menos JOINs complejos)")
print("✅ Optimizado para lectura (analytics)")
print("✅ Escalable (dimensiones pequeñas, facts grandes)")
print("✅ Flexible para agregaciones")
print("=" * 60)

## 8. Ejercicio: Dimensiones Lentamente Cambiantes (SCD Tipo 2)

Las dimensiones SCD Tipo 2 mantienen historial de cambios. Cuando un atributo cambia, se crea un nuevo registro en lugar de actualizar el existente.

In [ ]:
# Crear dimensión Cliente con SCD Tipo 2
# Mantendremos historial cuando cambie la región del cliente

dim_cliente_scd2 = pd.DataFrame({
    'cliente_id': [1, 1, 2, 3, 3, 4],
    'nombre': ['Cliente A', 'Cliente A', 'Cliente B', 'Cliente C', 'Cliente C', 'Cliente D'],
    'region': ['Norte', 'Sur', 'Este', 'Oeste', 'Norte', 'Este'],  # Cliente A y C cambiaron de región
    'fecha_inicio': ['2024-01-01', '2024-06-01', '2024-01-01', '2024-01-01', '2024-08-01', '2024-01-01'],
    'fecha_fin': ['2024-05-31', None, None, '2024-07-31', None, None],  # None = registro actual
    'es_actual': [False, True, True, False, True, True]
})

print("=== DIMENSIÓN CLIENTE CON SCD TIPO 2 ===")
print("\nHistorial de cambios:")
print(dim_cliente_scd2)

print("\n💡 Observa que:")
print("- Cliente A tiene 2 registros: uno histórico (Norte) y uno actual (Sur)")
print("- Cliente C tiene 2 registros: uno histórico (Oeste) y uno actual (Norte)")
print("- Cliente B y D solo tienen un registro (no han cambiado)")

# Filtrar solo registros actuales
clientes_actuales = dim_cliente_scd2[dim_cliente_scd2['es_actual'] == True]
print(f"\n✅ Registros actuales: {len(clientes_actuales)}")
print(clientes_actuales[['cliente_id', 'nombre', 'region', 'es_actual']])

### Simular cambio en SCD Tipo 2

Cuando un cliente cambia de región, en lugar de actualizar, creamos un nuevo registro:

In [ ]:
# Simular: Cliente B cambia de región de "Este" a "Sur" el 2024-09-01

# 1. Marcar registro anterior como histórico
dim_cliente_scd2.loc[
    (dim_cliente_scd2['cliente_id'] == 2) & (dim_cliente_scd2['es_actual'] == True),
    ['fecha_fin', 'es_actual']
] = ['2024-08-31', False]

# 2. Crear nuevo registro actual
nuevo_registro = pd.DataFrame({
    'cliente_id': [2],
    'nombre': ['Cliente B'],
    'region': ['Sur'],  # Nueva región
    'fecha_inicio': ['2024-09-01'],
    'fecha_fin': [None],
    'es_actual': [True]
})

dim_cliente_scd2 = pd.concat([dim_cliente_scd2, nuevo_registro], ignore_index=True)

print("=== DESPUÉS DEL CAMBIO ===")
print("\nHistorial completo de Cliente B:")
print(dim_cliente_scd2[dim_cliente_scd2['cliente_id'] == 2][['cliente_id', 'nombre', 'region', 'fecha_inicio', 'fecha_fin', 'es_actual']])

print("\n💡 Ventajas de SCD Tipo 2:")
print("✅ Mantiene historial completo de cambios")
print("✅ Permite análisis histórico preciso")
print("✅ Puedes ver cómo estaba un cliente en cualquier fecha")

## 9. Ejercicio: Identificar Granularidad

La granularidad define qué representa cada fila en una tabla de hechos. Es crucial definirla correctamente.

In [ ]:
print("=" * 70)
print("EJERCICIO: IDENTIFICAR GRANULARIDAD")
print("=" * 70)

print("\n📊 Ejemplo 1: Sistema de Ventas")
print("- Granularidad actual: Una fila por transacción/venta")
print("- Cada fila representa: Una venta de un producto a un cliente en una fecha")
print(f"- Ejemplo: {len(fact_ventas)} filas = {len(fact_ventas)} ventas individuales")
print("\n✅ Esta granularidad permite:")
print("  - Ver cada transacción individual")
print("  - Analizar patrones de compra detallados")
print("  - Agregar a cualquier nivel (día, mes, producto, cliente)")

print("\n" + "-" * 70)
print("\n📊 Ejemplo 2: ¿Qué pasaría con diferente granularidad?")

# Simular granularidad diaria (agregada)
ventas_diarias = fact_ventas.merge(
    dim_tiempo[['fecha_id', 'fecha']], on='fecha_id'
).groupby('fecha').agg({
    'total': 'sum',
    'cantidad': 'sum',
    'venta_id': 'count'
}).reset_index()
ventas_diarias.columns = ['fecha', 'total_diario', 'cantidad_total', 'num_transacciones']

print("\nGranularidad diaria (agregada):")
print(f"- Filas: {len(ventas_diarias)} (una por día)")
print(f"- Cada fila representa: Total de ventas de ese día")
print("\n⚠️ Con esta granularidad PERDERÍAS:")
print("  - Detalle de transacciones individuales")
print("  - Información de qué productos se vendieron")
print("  - Información de qué clientes compraron")
print("\n✅ Útil para:")
print("  - Reportes de alto nivel")
print("  - Análisis de tendencias diarias")
print("  - Reducir tamaño de datos")

print("\n" + "-" * 70)
print("\n💡 REGLA DE ORO:")
print("Define la granularidad MÁS BAJA que necesitas.")
print("Siempre puedes agregar, pero no puedes desagregar.")
print("=" * 70)

### Ejercicios de Granularidad

Para cada caso, identifica la granularidad apropiada:

In [ ]:
ejercicios_granularidad = """
📝 EJERCICIOS DE GRANULARIDAD:

1. Sistema de Inventario:
   - ¿Una fila por producto por día?
   - ¿Una fila por producto por hora?
   - ¿Una fila por movimiento de inventario?
   - 💡 Respuesta sugerida: Una fila por movimiento (más detalle)
     Luego puedes agregar a día/hora si es necesario

2. Sistema de Sensores IoT:
   - ¿Una fila por lectura de sensor?
   - ¿Una fila por minuto agregado?
   - ¿Una fila por hora agregada?
   - 💡 Respuesta sugerida: Depende del volumen
     - Si son pocos sensores: Una fila por lectura
     - Si son muchos: Una fila por minuto agregado

3. Sistema de E-commerce:
   - ¿Una fila por transacción?
   - ¿Una fila por producto en la transacción?
   - ¿Una fila por día?
   - 💡 Respuesta sugerida: Una fila por producto en la transacción
     Esto permite analizar productos individuales dentro de órdenes

4. Sistema de Ventas por Región:
   - ¿Una fila por venta?
   - ¿Una fila por región por día?
   - ¿Una fila por región por mes?
   - 💡 Respuesta sugerida: Una fila por venta
     Puedes agregar después, pero no puedes desagregar
"""

print(ejercicios_granularidad)
print("\n💡 Practica: Piensa en un caso de uso real y define su granularidad.")